In [35]:
from source.regency import Regency
from source.DQNAgent import DQNAgent
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')


seasons = 1
Game = Regency(IntDC=30)
clear_output()

for a in range(seasons):
    print(a)
    Game.random_events()
    Game.collect_regency_points()
    Game.domain_initiative()
    Game.collect_gold_bars()
    Game.maintenance_costs(Update=True)
    Game.take_domain_actions(None)
Game.clear_screen()
Game.Seasons[Game.Season]['Season']

,Regent,Player,Random Event,Regency Points,Initiative,Gold Bars,Revenue,Cost,Court
0,SM,False,Trade Matter,15,22,0,0,0,Dormant
1,EM,False,No Event,11,20,6,10,4,Bare
2,DW,False,No Event,71,20,33,11,11,Rich
3,HK,False,No Event,20,20,0,0,0,Dormant
4,EC,False,No Event,33,19,-1,10,11,Dormant
5,GTh,False,No Event,8,19,4,7,3,Bare
6,Ch,False,Unrest or Rebellion,28,18,0,0,0,Dormant
7,Rg,False,No Event,22,17,0,0,0,Dormant
8,HA,False,No Event,28,17,12,21,9,Average
9,MB,False,Corruption/Crime,18,17,11,15,4,Bare


In [36]:
import pandas as pd
df = pd.concat([pd.concat([Game.Seasons[a]['Actions'][b+1] for b in range(3)]) for a in range(seasons)])

In [37]:
set(df['Action'])

{'Lieutenant',
 'agitiate_for_friend',
 'declare_war',
 'decree_asset_seizure',
 'decree_general',
 'diplomacy_trade_agreement',
 'disband_army',
 'disband_mercenaries',
 'espionage_assassination',
 'espionage_corruption',
 'espionage_diplomatic_details',
 'espionage_discover_troop_movements',
 'espionage_heresy',
 'espionage_intrigue',
 'grant',
 'muster_army',
 'muster_levies',
 'muster_mercenaries'}

In [40]:
self=Game
import pandas as pd
Target

In [42]:
A = self.Relationships[self.Relationships['Regent']==Regent].copy()
A = A[A['Other']==Target]
B = self.Relationships[self.Relationships['Regent']==Target].copy()
B = B[B['Other']==Regent]
temp = pd.concat([A, B])
if np.sum(temp['Trade Permission']) > 0:
    # check for provences
    temp = pd.merge(temp[['Regent']],self.Provences, on='Regent', how='left')

NameError: name 'Target' is not defined